In [6]:
%pip install trio

  Using cached trio-0.27.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached trio-0.27.0-py3-none-any.whl (481 kB)
Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
import trio

In [4]:
print('hello')

hello


In [13]:
async def sleepy():
  print("yawn!")
  await trio.sleep(1)
  print("Awake!")

async def sleep_twice():
  await sleepy()
  await sleepy()

import time
start = time.time()
trio.run(sleep_twice)
end = time.time()
print(end - start)

yawn!
Awake!
yawn!
Awake!
2.0133891105651855


In [15]:
async def sleep_concurrently():
  async with trio.open_nursery() as nurssery:
    nurssery.start_soon(sleepy)
    nurssery.start_soon(sleepy)

In [16]:
start = time.time()
trio.run(sleep_concurrently)
end = time.time()
print(end - start)

yawn!
yawn!
Awake!
Awake!
1.0052180290222168


In [ ]:
from errno import EADDRNOTAVAIL
from functools import partial
from itertools import count
import trio
import socket

async def peer(SRC, DEST):
    counter = count(start=1)
    async def sender(stream, n):
        print(f"sender{n}@{SRC}: started!")
        while True:
            data = bytes(f"Hello from {n}@{SRC}", "utf8")
            print(f"sender{n}@{SRC}: sending {data!r}")
            await stream.send_all(data)
            await trio.sleep(1)

    async def receiver(stream, n):
        print(f"receiver{n}@{SRC}: started!")
        async for data in stream:
            print(f"receiver{n}@{SRC}: got data {data!r}")
        print(f"receiver{n}@{SRC}: connection closed")

    async with trio.open_nursery() as nursery:
        async def run(method, connection: trio.SocketStream):
            count = next(counter)
            print(f"peer@{SRC} got connection{count} from {method}() with {connection.socket.getpeername()}")
            async with connection:
                async with trio.open_nursery() as nursery:
                    print(f"peer@{SRC}: spawning sender...")
                    nursery.start_soon(sender, connection, count)

                    print(f"peer@{SRC}: spawning receiver...")
                    nursery.start_soon(receiver, connection, count)

        print(f"peer: listening at {SRC}")
        servers = await trio.open_tcp_listeners(SRC[1], host=SRC[0])
        servers[0].socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        servers[0].socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)
        await nursery.start(trio.serve_listeners, partial(run, "listen"), servers)

        print(f"peer: connecting from {SRC} to {DEST}")
        client = trio.socket.socket()
        client.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        client.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEPORT, 1)
        await client.bind(address=SRC)
        try:
            await client.connect(address=DEST)
        except OSError as err:
            if err.errno != EADDRNOTAVAIL:
                raise
            # the other client was faster than us
            print(f"peer@{SRC}: {err.strerror}")
        else:
            await run('connect', trio.SocketStream(client))

async def main():
    async with trio.open_nursery() as nursery:
        a = ("127.0.0.1", 12346)
        b = ("127.0.0.1", 54322)
        c = ("127.0.0.1", 54342)
        nursery.start_soon(peer, a, b)
        nursery.start_soon(peer, b, a)
        nursery.start_soon(peer, a, c)

trio.run(main)

peer: listening at ('127.0.0.1', 54322)
peer: listening at ('127.0.0.1', 12346)
peer: connecting from ('127.0.0.1', 54322) to ('127.0.0.1', 12346)
peer: connecting from ('127.0.0.1', 12346) to ('127.0.0.1', 54322)
peer@('127.0.0.1', 12346) got connection1 from connect() with ('127.0.0.1', 54322)
peer@('127.0.0.1', 12346): spawning sender...
peer@('127.0.0.1', 12346): spawning receiver...
peer@('127.0.0.1', 54322) got connection1 from connect() with ('127.0.0.1', 12346)
peer@('127.0.0.1', 54322): spawning sender...
peer@('127.0.0.1', 54322): spawning receiver...
receiver1@('127.0.0.1', 54322): started!
sender1@('127.0.0.1', 54322): started!
sender1@('127.0.0.1', 54322): sending b"Hello from 1@('127.0.0.1', 54322)"
receiver1@('127.0.0.1', 12346): started!
sender1@('127.0.0.1', 12346): started!
sender1@('127.0.0.1', 12346): sending b"Hello from 1@('127.0.0.1', 12346)"
receiver1@('127.0.0.1', 12346): got data b"Hello from 1@('127.0.0.1', 54322)"
receiver1@('127.0.0.1', 54322): got data b"H

  + Exception Group Traceback (most recent call last):
  |   File "/Users/karthiksridhar/anaconda3/envs/capstone/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |     ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/var/folders/ky/vwqvfz114yj_cz29njlnph180000gn/T/ipykernel_62351/1079804873.py", line 63, in <module>
  |     trio.run(main)
  |     ~~~~~~~~^^^^^^
  |   File "/Users/karthiksridhar/anaconda3/envs/capstone/lib/python3.13/site-packages/trio/_core/_run.py", line 2407, in run
  |     raise runner.main_task_outcome.error
  |   File "/var/folders/ky/vwqvfz114yj_cz29njlnph180000gn/T/ipykernel_62351/1079804873.py", line 57, in main
  |     async with trio.open_nursery() as nursery:
  |                ~~~~~~~~~~~~~~~~~^^
  |   File "/Users/karthiksridhar/anaconda3/envs/capstone/lib/python3.13/site-packages/trio/_core/_run.py", line 1039, in __aexit__
  |     raise combine